# 6.3 填充和步幅
- **目录**
  - 6.3.1 填充
  - 6.3.2 步幅


在前面的例子 图6.2.1中，输入的高度和宽度都为$3$，卷积核的高度和宽度都为$2$，生成的输出表征的维数为$2\times2$。
正如我们在 6.2节中所概括的那样，**假设输入形状为$n_h\times n_w$，卷积核形状为$k_h\times k_w$，那么输出形状将是$(n_h-k_h+1) \times (n_w-k_w+1)$。
因此，卷积的输出形状取决于输入形状和卷积核的形状。**

还有什么因素会影响输出的大小呢？本节我们将介绍**填充（padding）**和**步幅（stride）**。假设以下情景：
有时，在应用了连续的卷积之后，我们最终得到的输出远小于输入大小。这是由于卷积核的宽度和高度通常大于$1$所导致的。比如，一个$240 \times 240$像素的图像，经过$10$层$5 \times 5$的卷积后，将减少到$200 \times 200$像素。如此一来，原始图像的**边界丢失了许多有用信息**。而**填充**是解决此问题最有效的方法。
有时，我们可能希望大幅降低图像的宽度和高度。例如，如果我们发现原始的输入分辨率十分冗余。**步幅**则可以在这类情况下提供帮助。

- **要点：**
  - **输出形状的计算**：给定输入形状$(n_h \times n_w)$和卷积核形状$(k_h \times k_w)$时，输出形状被计算为$(n_h - k_h + 1) \times (n_w - k_w + 1)$。
  - **填充（Padding）**：为了避免连续卷积操作导致输出尺寸急剧减小，可以在输入数据的周围添加填充。
  - **步幅（Stride）**：如果需要降低输出的宽度和高度，可以增加卷积操作的步幅。



----------
- **说明：为何原始图像的边界会丢失许多有用信息？**
  - **卷积操作的本质**
    - **卷积核滑动**：卷积核从图像的左上角开始，逐步向右下角移动。每一步，核与图像的对应部分进行元素乘积并求和，生成输出特征图的一个像素。
    - **边界效应**：当卷积核接近图像边界时，核的一部分会滑出图像边界。这意味着核不能完全覆盖图像的边界像素，因此这些像素的贡献在输出特征图中被减少或完全丢失。
  - **边界信息的丢失**
    - **信息量减少**：在没有填充的情况下，每次卷积后，输出图像的大小会比输入小。具体来说，如果使用的是`n x n`的卷积核，每次卷积后，输出图像的边界会向内收缩`(n-1)/2`个像素（对于奇数大小的核）。
    - **边界像素的利用率**：图像的中心像素会在多次卷积中被多次使用（因为卷积核可以完全覆盖它们），而边界像素则只在卷积核刚好覆盖到它们时被使用。这导致边界信息在深层网络中**被稀释或丢失**。
  - **为什么边界信息重要**
    - **空间信息**：图像的边界通常包含了关于物体边缘、背景与前景分界等重要空间信息，这些信息对于图像识别、分割等任务至关重要。
    - **完整性**：对于一些应用，如图像分割或需要精确定位的任务，丢失边界信息可能导致性能下降，因为模型可能无法准确地理解或重建图像的完整结构。
    - **上下文理解**：边界信息可以提供关于图像中物体相互关系的上下文，这对于理解整个场景是必要的。
  - **填充的作用**
    - **保持尺寸**：通过在图像周围添加额外的像素（通常是零，也可以是其他值或方法），可以确保卷积后输出的尺寸与输入相同或更可控。
    - **保留边界信息**：填充使得卷积核在图像边界也能产生有效的输出，从而保留了原本可能丢失的边界信息。
  - 因此，说“原始图像的边界丢失了许多有用信息”是因为在没有填充的卷积操作中，图像的边界像素由于卷积核的滑动特性而无法**充分**参与到特征提取的过程中，导致这些区域的信息在网络的更深层中被忽视或丢失。
  - 填充技术通过在图像边界添加像素，解决了这个问题，使得网络能够利用到图像的全部信息。   
-----------

## 6.3.1 填充

如上所述，在应用多层卷积时，我们**常常丢失边缘像素。**
由于我们通常使用小卷积核，因此对于任何单个卷积，我们可能只会丢失几个像素。
但随着我们应用许多连续卷积层，累积丢失的像素数就多了。
解决这个问题的简单方法即为**填充（padding）：在输入图像的边界填充元素（通常填充元素是$0$）。**
例如，在 图6.3.1中，我们将$3 \times 3$输入填充到$5 \times 5$，那么它的输出就增加为$4 \times 4$。阴影部分是第一个输出元素以及用于输出计算的输入和核张量元素：
$0\times0+0\times1+0\times2+0\times3=0$。
<center>
    <img src="../img/conv-pad.svg" alt="带填充的二维互相关">
</center>
<center>图6.3.1 带填充的二维互相关</center>

通常，如果我们添加$p_h$行填充（大约一半在顶部，一半在底部）和$p_w$列填充（左侧大约一半，右侧一半），则输出形状将为

$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1) \tag{6.3.1}$$

这意味着输出的高度和宽度将分别增加$p_h$和$p_w$。

**在许多情况下，我们需要设置$p_h=k_h-1$和$p_w=k_w-1$，使输入和输出具有相同的高度和宽度。**
这样可以在构建网络时更容易地预测每个图层的输出形状。

假设$k_h$是**奇数**，我们将在高度的两侧（即顶部和底部）填充$p_h/2$行。

如果$k_h$是**偶数**，则一种可能性是在输入顶部填充$\lceil p_h/2\rceil$（即$ p_h/2$**上取整**）行，在底部填充$\lfloor p_h/2\rfloor$（$ p_h/2$**下取整**）行。同理，我们填充宽度的两侧。

**卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7。
选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。**

此外，使用奇数的核大小和填充大小也提供了书写上的便利。对于任何二维张量`X`，当满足：
1. 卷积核的大小是奇数；
2. 所有边的填充行数和列数相同；
3. 输出与输入具有相同高度和宽度
则可以得出：**输出`Y[i, j]`是通过以输入`X[i, j]`为中心，与卷积核进行互相关计算得到的**。

比如，在下面的例子中，我们创建一个高度和宽度为3的二维卷积层，并(**在所有侧边填充1个像素**)。给定高度和宽度为8的输入，则输出的高度和宽度也是8。

- **要点：**
  - 在应用多层卷积时，边缘像素常常会丢失。为了解决这个问题，我们可以使用填充（padding）方法，在输入图像的边界填充元素（通常为0）。
  - 填充可以使输出的高度和宽度增加，从而减少连续卷积层中累积丢失的像素数。
  - 通常需要设置填充大小，以使输入和输出具有相同的高度和宽度，这样可以更容易地预测每个图层的输出形状。
  - 卷积神经网络中卷积核的高度和宽度通常为奇数（如1、3、5或7），这样在保持空间维度的同时，可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。
  - 使用奇数核大小和填充大小还提供了书写上的便利，当满足特定条件时，输出可以通过以输入为中心与卷积核进行互相关计算得到。

In [11]:
import torch
from torch import nn


# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1，
    '''
    torch的通道是第二维，Tensorflow是在第四维，第一维都是批量大小。
    注意这里的+操作是将两个元组连接合并，而不是进行算术运算，
    比如(1,1)+(8,8)=(1,1,8,8)
    '''
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

'''
请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列。
padding参数的赋值不同，填充的方式也不同，此处=1表示上下左右各填充一行或一列。
由于对输入张量进行填充，因此填充后的输入张量X的高和宽变成(10,10)。
那么输出张量的大小将变成(10-3+1)*(10-3+1)=(8,8)。
或者按照上述公式进行计算等于(8-3+2+1)，也是(8,8)。
'''
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [8]:
## 多个元组连接
(1,1)+(4,5)

(1, 1, 4, 5)

- 当卷积核的高度和宽度不同时，可以**填充不同的高度和宽度**，使输出和输入具有相同的高度和宽度。
- 在如下示例中，使用高度为5，宽度为3的卷积核，高度和宽度两边的填充分别为2和1。


In [12]:
'''
padding=(2, 1)表示，顶部和底部各填充两行，左边右边各填充一列，
那么输入张量的形状就变成(12,10),核的大小为(5,3)，按照公式6.3.1进行计算，
最后输出的大小仍是(8,8)。
'''
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## 6.3.2 步幅

在计算互相关时，卷积窗口从输入张量的左上角开始，向下、向右滑动。
在前面的例子中，我们**默认每次滑动一个元素**。
但是，有时候为了高效计算或是**缩减采样次数**，卷积窗口可以跳过中间位置，每次滑动多个元素。

我们将每次滑动元素的数量称为**步幅（stride）**。到目前为止，我们只使用过高度或宽度为$1$的步幅，那么如何使用较大的步幅呢？
图6.3.2是垂直步幅为$3$，水平步幅为$2$的二维互相关运算。
着色部分是输出元素以及用于输出计算的输入和卷积核张量元素：$0\times0+0\times1+1\times2+2\times3=8$、$0\times0+6\times1+0\times2+0\times3=6$。

可以看到，为了计算输出中第一列的第二个元素和第一行的第二个元素，卷积窗口分别向下滑动三行和向右滑动两列。但是，当卷积窗口继续向右滑动两列时，没有输出，因为输入元素无法填充窗口（除非我们添加另一列填充）。


<center>
    <img src="../img/conv-stride.svg" alt="垂直步幅为 $3$，水平步幅为 $2$ 的二维互相关运算">
</center>
<center>
    图6.3.2 垂直步幅为3 ，水平步幅为2 的二维互相关运算
</center>

通常，当垂直步幅为$s_h$、水平步幅为$s_w$时，输出形状为

$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.\tag{6.3.2}$$

如果我们设置了$p_h=k_h-1$和$p_w=k_w-1$，则输出形状将简化为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。
更进一步，如果输入的高度和宽度可以被垂直和水平步幅整除，则输出形状将为$(n_h/s_h) \times (n_w/s_w)$。
- **要点：**
  - 步幅（stride）是卷积窗口在输入张量上滑动时每次移动的元素数量。
  - 使用较大的步幅可以提高计算效率，减少采样次数，并降低输出特征图的尺寸。
  - 垂直步幅为 $s_h$，水平步幅为 $s_w$。
  - 当使用较大的步幅时，输出形状的计算公式为：$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor$。
  - 如果设置了 $p_h=k_h-1$ 和 $p_w=k_w-1$，则输出形状将简化为 $\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。
  - 当输入的高度和宽度能被垂直和水平步幅整除时，输出形状将为 $(n_h/s_h) \times (n_w/s_w)$。
  - 通过设置高度和宽度的步幅为 2，可以将输入的高度和宽度减半。



- 下面**将高度和宽度的步幅设置为2**，从而将输入的高度和宽度减半。


In [15]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

- 一个稍微复杂的例子。

In [16]:
'''
根据公式6.3.2进行计算:
输出的h = floor((8 - 3 + 0 + 3)/3) = 2
输出的w = floor((8 - 5 + 1 + 4)/4) = 2
'''
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
## comp_conv2d函数挤压第1和第2维
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

- 当输入高度和宽度两侧的填充数量分别为$p_h$和$p_w$时，我们称之为填充$(p_h, p_w)$。当$p_h = p_w = p$时，填充是$p$。
- 同理，当高度和宽度上的步幅分别为$s_h$和$s_w$时，我们称之为步幅$(s_h, s_w)$。
- 当步幅为$s_h = s_w = s$时，步幅为$s$。默认情况下，填充为0，步幅为1。
- 在实践中，很少使用不一致的步幅或填充，一般情况下都是$p_h = p_w$和$s_h = s_w$。

## 小结

* 填充可以增加输出的高度和宽度。这常用来使输出与输入具有相同的高和宽。
* 步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的$1/n$（$n$是一个大于$1$的整数）。
* 填充和步幅可用于有效地调整数据的维度。